In [1]:
# !pip install pytorch-tabnet

In [2]:
import os
from glob import glob
import math

import numpy as np
import pandas as pd
import re

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.metrics import Metric

import torch.nn.functional as F
import torch.optim as optim
# from datasets import Dataset
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler # 표준화 패키지 라이브러리
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import pickle
from tqdm import tqdm
import random
from warnings import filterwarnings
import gc
from collections import Counter
import matplotlib.pyplot as plt

filterwarnings('ignore')
device = torch.device('cpu')
os.getcwd()

'/Users/louan/Documents/projects/Aiffel/Hackathon/geodata-con/geo-con/models/RT_TabNet'

In [3]:
import data_integration as di

# Data Load

In [4]:
# 전체 file list 만들기
path = '../../../3W/dataset/' # the path of 3W/dataset/
for event in range(9):
    file_lst = []
    full_path = path + f'{event}/*.csv'
    print(full_path)
    file_lst.extend(sorted(glob(full_path)))
    globals()['df_event_{}'.format(event)] = di.dt_integration(file_lst[:20])
    globals()['df_event_{}'.format(event)].drop(columns = ['timestamp', 'event_type', 'P-JUS-CKGL', 'T-JUS-CKGL', 'QGL', 'instance_type'], inplace = True)
    globals()['df_event_{}'.format(event)].reset_index(inplace=True, drop = True)

../../../3W/dataset/0/*.csv
../../../3W/dataset/1/*.csv
../../../3W/dataset/2/*.csv
../../../3W/dataset/3/*.csv
../../../3W/dataset/4/*.csv
../../../3W/dataset/5/*.csv
../../../3W/dataset/6/*.csv
../../../3W/dataset/7/*.csv
../../../3W/dataset/8/*.csv


In [5]:
# test
df_event_3

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,class,id_label
0,19426080.0,12508490.0,121.7742,4034498.0,86.99649,3,SIMULATED_00001_3
1,19427790.0,12510300.0,121.7738,4034516.0,86.99896,3,SIMULATED_00001_3
2,19429510.0,12512120.0,121.7734,4034534.0,87.00143,3,SIMULATED_00001_3
3,19431230.0,12513930.0,121.7733,4034553.0,87.00390,3,SIMULATED_00001_3
4,19432940.0,12515750.0,121.7730,4034570.0,87.00637,3,SIMULATED_00001_3
...,...,...,...,...,...,...,...
1125975,17411620.0,10125210.0,105.6147,4083094.0,45.40679,3,SIMULATED_00020_3
1125976,17411820.0,10125280.0,105.6101,4083095.0,45.41088,3,SIMULATED_00020_3
1125977,17412020.0,10125350.0,105.6054,4083094.0,45.41497,3,SIMULATED_00020_3
1125978,17412230.0,10125420.0,105.6007,4083093.0,45.41905,3,SIMULATED_00020_3


In [6]:
def interplolate_without_class(df, method='linear'):
    '''
    input (df) : 라벨 단위 observation_df
    output (df) : 라벨 단위 observation_df
    기본 보간 방법은 선형 보간
    '''
    columns = df.columns
    columns = columns.drop('class')
    class_observations = df['class']

    df = df[columns].interpolate(method = method)
    df['class'] = class_observations
    return df

In [17]:
threshold = 10
events = 9 # 0부터 8번까지의 이벤트를 따로따로 저장함

before_interpolation_description = []
after_interpolation_description = []
observation_nums_lst = []

for event in range(event):
    #1. 인스턴스 별 칼럼의 결측 비율 조사
    df = globals()['df_event_{}'.format(event)]
    missing_proportion_df = di.missing_data_proportion(df)

    #2. Threshold 이상의 observation만 남기고 나머지 observation 제거
    index = missing_proportion_df[(missing_proportion_df > threshold).sum(axis=1) == 0].index
    print(index)
    df = df[df['id_label'].isin(index)]
    print(f"{event} 라벨의 최종 관측 개수: {len(df)}")

    #3. 라벨 제외한 df 보간
    globals()['df_event_{}'.format(event)] = interplolate_without_class(df, method = 'linear')
    


In [14]:
# path = '../../../data/*.pkl'

# file_lst = sorted(glob(path))[1:]
# file_lst[6] = '../../../data/pre_processed_split_0tpt_data_6.pkl'
# print(file_lst)

['../../../data/pre_processed_split_data_0.pkl', '../../../data/pre_processed_split_data_1.pkl', '../../../data/pre_processed_split_data_2.pkl', '../../../data/pre_processed_split_data_3.pkl', '../../../data/pre_processed_split_data_4.pkl', '../../../data/pre_processed_split_data_5.pkl', '../../../data/pre_processed_split_0tpt_data_6.pkl', '../../../data/pre_processed_split_data_7.pkl', '../../../data/pre_processed_split_data_8.pkl']


In [19]:
# # ModuleNotFoundError: No module named 'pandas.core.indexes.numeric' using Metaflow
# # This issue is caused by the new Pandas 2.0.0 release => pip install "pandas<2.0.0"
# # https://stackoverflow.com/questions/75953279/modulenotfounderror-no-module-named-pandas-core-indexes-numeric-using-metaflo
# for i,path in enumerate(file_lst):
#     print(i, path)
#     with open(path, 'rb') as f:
#         globals()['df_event_{}'.format(i)] = pickle.load(f)
#         globals()['df_event_{}'.format(i)].reset_index(inplace = True, drop = True)

0 ../../../data/pre_processed_split_data_0.pkl
1 ../../../data/pre_processed_split_data_1.pkl
2 ../../../data/pre_processed_split_data_2.pkl
3 ../../../data/pre_processed_split_data_3.pkl
4 ../../../data/pre_processed_split_data_4.pkl
5 ../../../data/pre_processed_split_data_5.pkl
6 ../../../data/pre_processed_split_0tpt_data_6.pkl
7 ../../../data/pre_processed_split_data_7.pkl
8 ../../../data/pre_processed_split_data_8.pkl


## Outlier 제거
- 참고: https://stackoverflow.com/questions/35827863/remove-outliers-in-pandas-dataframe-using-percentiles

In [19]:
# Add delta and remove outlier
def add_delta(data_frame):
    for column in ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']:
        column_pctchg = column+'_pctchg'
        data_frame[column_pctchg] = data_frame.groupby(['id_label'])[column].pct_change()
        data_frame[column_pctchg][data_frame[column_pctchg].isna()] = 0
    return data_frame

def remove_outlier(df):
    target_cols = ['P-PDG', 'P-TPT', 'T-TPT','P-MON-CKP', 'T-JUS-CKP',]
    df = df[df['id_label'].str.startswith('WELL')]
    df = add_delta(df)
    Q1 = df[target_cols].quantile(0.25)
    Q3 = df[target_cols].quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df[target_cols] < (Q1 - 1.5 * IQR)) | (df[target_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df

In [20]:
for i in range(9):
    print(f'event_num: {i}')
    df_event_0_out = remove_outlier(df_event_0)
    df_event_1_out = remove_outlier(df_event_1)
    df_event_2_out = remove_outlier(df_event_2)
    df_event_3_out = remove_outlier(df_event_3)
    df_event_4_out = remove_outlier(df_event_4)
    df_event_5_out = remove_outlier(df_event_5)
    df_event_6_out = remove_outlier(df_event_6)
    df_event_7_out = remove_outlier(df_event_7)
    df_event_8_out = remove_outlier(df_event_8)

0
1
2
3
4
5
6
7
8


# Train Test split

In [21]:
# extract the list of each data type(WELL, SIMULATED, DRAWN)
instance_dict = {'WELL': [], 'SIMULATED': [], 'DRAWN': []}
for i in range(9):
  instance_lst = list(globals()['df_event_{}'.format(i)]['id_label'].unique())
  for instance in instance_lst:
    if instance.startswith('WELL'):
      instance_dict['WELL'].extend([instance])
    elif instance.startswith('SIMULATED'):
      instance_dict['SIMULATED'].extend([instance])
    elif instance.startswith('DRAWN'):
      instance_dict['DRAWN'].extend([instance])

In [22]:
# check the number of events in 'WELL' data
class_lst = []
for i in instance_dict['WELL']:
  class_lst.append(i.split('_')[-1])
Counter(class_lst)

Counter({'0': 20, '4': 20, '7': 5, '2': 4})

In [23]:
# split train and test instances
def split_train_test_instance(instance_dict, dTypes = ['WELL', 'SIMULATED', 'DRAWN'],train_ratio = 0.6, test_ratio = 0.2):
    total_len = 0
    simul_drawn_lst = []
    for dType in dTypes:
        total_len += len(instance_dict[dType])
        simul_drawn_lst += instance_dict[dType]
    well_len = len(instance_dict['WELL'])

    num_of_test = int(total_len*test_ratio)
    num_of_val = int(total_len*(1-train_ratio-test_ratio))
    num_of_train = total_len - num_of_test - num_of_val

    # extract test_lst first(Only WELL data)
    test_lst = random.sample(instance_dict['WELL'], num_of_test)
    
    # then make whole lst --> shuffle lst --> split train validation set
    rest_well_lst = [i for i in instance_dict['WELL'] if i not in test_lst]
    total_lst = rest_well_lst + simul_drawn_lst
    random.shuffle(total_lst)

    train_lst = total_lst[:num_of_train]
    val_lst = total_lst[num_of_train:]

    return train_lst, val_lst, test_lst

In [24]:
train_ratio = 0.7
test_ratio = 0.15
val_ratio = 1 - train_ratio - test_ratio
dTypes = ['WELL', 'SIMULATED', 'DRAWN']
train_lst, val_lst, test_lst = split_train_test_instance(instance_dict, dTypes = dTypes, train_ratio = train_ratio, test_ratio = test_ratio)

print(len(train_lst), len(val_lst), len(test_lst), sep = '\n')
print('total number of lst:', len(instance_dict['WELL'])+len(instance_dict['DRAWN'])+len(instance_dict['SIMULATED']))
print('total number of lst after removing duplicated:', len(set(train_lst + val_lst + test_lst)))
print('train:', sorted(set([i.split('_')[-1] for i in train_lst])), 'val:', sorted(set([i.split('_')[-1] for i in val_lst])), 'test:', sorted(set([i.split('_')[-1] for i in test_lst])), sep = '\n')

123
75
26
total number of lst: 175
total number of lst after removing duplicated: 175
train:
['0', '1', '2', '3', '4', '5', '6', '7', '8']
val:
['0', '1', '2', '3', '4', '5', '6', '7', '8']
test:
['0', '2', '4', '7']


In [25]:
# 참고: https://rfriend.tistory.com/590
def make_empty_df(pca_n_components):
    pca_lst = ['pca_1', 'pca_2','pca_3', 'pca_4', 'pca_5']
    columns = ['P-PDG', 'P-PDG_std','P-PDG_pctchg_mean', 'P-PDG_pctchg_std',
               'P-TPT', 'P-TPT_std', 'P-TPT_pctchg_mean', 'P-TPT_pctchg_std',
               'T-TPT', 'T-TPT_std', 'T-TPT_pctchg_mean', 'T-TPT_pctchg_std',
               'P-MON-CKP', 'P-MON-CKP_std', 'P-MON-CKP_pctchg_mean', 'P-MON-CKP_pctchg_std',
               'T-JUS-CKP', 'T-JUS-CKP_std', 'T-JUS-CKP_pctchg_mean', 'T-JUS-CKP_pctchg_std',
               'class',]
    columns.extend(pca_lst[:pca_n_components])
    data_frame = pd.DataFrame(columns=columns)
    return data_frame


def apply_window(data_frame, window_size = 5000, overlap_ratio = 0.2, pca_n_components = 2):
    if pca_n_components:
        pca = PCA(n_components=pca_n_components)
    
    new_df = make_empty_df(pca_n_components)
    for n in tqdm(range(int(len(data_frame)/(window_size*(1-overlap_ratio))))):
        try:
            if int(window_size*(n+1) - window_size*overlap_ratio*n) <= len(data_frame):
                # print(window_size*n*(1-overlap_ratio),window_size*(n+1) - window_size*overlap_ratio*n)
                sliced_df = data_frame[int(window_size*n*(1-overlap_ratio)):int(window_size*(n+1) - window_size*overlap_ratio*n)]
                pca_data = pca.fit_transform(sliced_df.drop( ['id_label','class'], axis=1))
                if any(sliced_df['class'] != 0):
                    class_label =  int(sliced_df['class'].value_counts().index[sliced_df['class'].value_counts().index < 10][0]) # 기본 모드
                    
                    # class_label = 1
                else:
                    # continue
                    class_label = 0
                new_df = pd.concat([new_df,pd.DataFrame({'P-PDG':[sliced_df['P-PDG'].mean()],
                                                        'P-PDG_std':[sliced_df['P-PDG'].std()],
                                                        'P-PDG_pctchg_mean':[sliced_df['P-PDG_pctchg'].mean()],
                                                        'P-PDG_pctchg_std':[sliced_df['P-PDG_pctchg'].std()],
                                                        'P-TPT':[sliced_df['P-TPT'].mean()],
                                                        'P-TPT_std':[sliced_df['P-TPT'].std()],
                                                        'P-TPT_pctchg_mean':[sliced_df['P-TPT_pctchg'].mean()],
                                                        'P-TPT_pctchg_std':[sliced_df['P-TPT_pctchg'].std()],
                                                        'T-TPT':[sliced_df['T-TPT'].mean()],
                                                        'T-TPT_std':[sliced_df['T-TPT'].std()],
                                                        'T-TPT_pctchg_mean':[sliced_df['T-TPT_pctchg'].mean()],
                                                        'T-TPT_pctchg_std':[sliced_df['T-TPT_pctchg'].std()],
                                                        'P-MON-CKP':[sliced_df['P-MON-CKP'].mean()],
                                                        'P-MON-CKP_std':[sliced_df['P-MON-CKP'].std()],
                                                        'P-MON-CKP_pctchg_mean':[sliced_df['P-MON-CKP_pctchg'].mean()],
                                                        'P-MON-CKP_pctchg_std':[sliced_df['P-MON-CKP_pctchg'].std()],
                                                        'T-JUS-CKP':[sliced_df['T-JUS-CKP'].mean()],
                                                        'T-JUS-CKP_std':[sliced_df['T-JUS-CKP'].std()],
                                                        'T-JUS-CKP_pctchg_mean':[sliced_df['T-JUS-CKP_pctchg'].mean()],
                                                        'T-JUS-CKP_pctchg_std':[sliced_df['T-JUS-CKP_pctchg'].std()],
                                                        'class': class_label,
                                                        'pca_1':[pca_data[:,0].mean()],
                                                        'pca_2':[pca_data[:,1].mean()],
                                                        #'pca_3':[pca_data.iloc[:,2].mean()],
                                                        })])

        except:
            continue
    return new_df

def build_dataset(data_frame, train_lst, val_lst, test_lst, window_size = 5000, overlap_ratio = 0.2, pca_n_components = 2):
  gc.collect()
  train_data_frame = data_frame[data_frame['id_label'].isin(train_lst)]
  val_data_frame = data_frame[data_frame['id_label'].isin(val_lst)]
  test_data_frame = data_frame[data_frame['id_label'].isin(test_lst)]

#   train_data_frame = remove_outlier(train_data_frame)

  train_df = apply_window(train_data_frame, window_size = window_size, overlap_ratio = overlap_ratio)
  val_df = apply_window(val_data_frame, window_size = window_size, overlap_ratio = overlap_ratio)
  test_df = apply_window(test_data_frame, window_size = window_size, overlap_ratio = overlap_ratio)
  return train_df, val_df, test_df

In [26]:
# applying window
window_size = 1200
window_size_short = window_size
window_size_long =  window_size
pca_n_components = 2
overlap_ratio = 0.95

df_event_0_window_train, df_event_0_window_val, df_event_0_window_test = build_dataset(df_event_0_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_1_window_train, df_event_1_window_val, df_event_1_window_test = build_dataset(df_event_1_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_2_window_train, df_event_2_window_val, df_event_2_window_test = build_dataset(df_event_2_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_3_window_train, df_event_3_window_val, df_event_3_window_test = build_dataset(df_event_3_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_4_window_train, df_event_4_window_val, df_event_4_window_test = build_dataset(df_event_4_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_5_window_train, df_event_5_window_val, df_event_5_window_test = build_dataset(df_event_5_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_6_window_train, df_event_6_window_val, df_event_6_window_test = build_dataset(df_event_6_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_7_window_train, df_event_7_window_val, df_event_7_window_test = build_dataset(df_event_7_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_8_window_train, df_event_8_window_val, df_event_8_window_test = build_dataset(df_event_8_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
gc.collect()


100%|██████████| 3342/3342 [00:19<00:00, 172.58it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 261/261 [00:00<00:00, 2054.59it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1138/1138 [00:07<00:00, 156.09it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 5730/5730 [00:28<00:00, 200.01it/s]
0it [00:00, ?it/s]
100%|██████████| 4513/4513 [00:20<00:00, 215.88it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


0

In [27]:
# Downsampling for train dataset
merged_df_train =  pd.concat([df_event_0_window_train,
                              df_event_1_window_train,
                              df_event_2_window_train,
                              df_event_3_window_train,
                              df_event_4_window_train,
                              df_event_5_window_train,
                              df_event_6_window_train,
                              df_event_7_window_train,
                              df_event_8_window_train])

merged_df_val =  pd.concat([df_event_0_window_val,
                            df_event_1_window_val,
                            df_event_2_window_val,
                            df_event_3_window_val,
                            df_event_4_window_val,
                            df_event_5_window_val,
                            df_event_6_window_val,
                            df_event_7_window_val,
                            df_event_8_window_val])

merged_df_test =  pd.concat([df_event_0_window_test,
                             df_event_1_window_test,
                             df_event_2_window_test,
                             df_event_3_window_test,
                             df_event_4_window_test,
                             df_event_5_window_test,
                             df_event_6_window_test,
                             df_event_7_window_test,
                             df_event_8_window_test])


In [28]:
print(merged_df_train['class'].value_counts())
print(merged_df_val['class'].value_counts())
print(merged_df_test['class'].value_counts())

0    6094
4    1656
7      84
Name: class, dtype: int64
0    3260
4    1102
Name: class, dtype: int64
0    4579
4    1119
7      59
Name: class, dtype: int64


In [29]:
# P-PDG_pctchg_mean = -inf, 혹은 inf인 row 제외: train(14개), val(10개), test(13)
merged_df_train = merged_df_train[~merged_df_train['P-PDG_pctchg_std'].isna()]
merged_df_val = merged_df_val[~merged_df_val['P-PDG_pctchg_std'].isna()]
merged_df_test = merged_df_test[~merged_df_test['P-PDG_pctchg_std'].isna()]

In [30]:
features = ['P-PDG', 'P-PDG_std','P-PDG_pctchg_mean', 'P-PDG_pctchg_std',
            'P-TPT', 'P-TPT_std', 'P-TPT_pctchg_mean', 'P-TPT_pctchg_std',
            'T-TPT', 'T-TPT_std', 'T-TPT_pctchg_mean', 'T-TPT_pctchg_std',
            'P-MON-CKP', 'P-MON-CKP_std', 'P-MON-CKP_pctchg_mean', 'P-MON-CKP_pctchg_std',
            'T-JUS-CKP', 'T-JUS-CKP_std', 'T-JUS-CKP_pctchg_mean', 'T-JUS-CKP_pctchg_std',
            'pca_1', 'pca_2']
X_train_pre, y_train = merged_df_train.loc[:,features], merged_df_train['class']
X_train_pre

,P-PDG,P-PDG_std,P-PDG_pctchg_mean,P-PDG_pctchg_std,P-TPT,P-TPT_std,P-TPT_pctchg_mean,P-TPT_pctchg_std,T-TPT,T-TPT_std,...,P-MON-CKP,P-MON-CKP_std,P-MON-CKP_pctchg_mean,P-MON-CKP_pctchg_std,T-JUS-CKP,T-JUS-CKP_std,T-JUS-CKP_pctchg_mean,T-JUS-CKP_pctchg_std,pca_1,pca_2
0,0.000000e+00,0.000000e+00,0.0,0.0,9.994886e+06,10897.730268,-8.660628e-07,0.000007,119.016257,1.142084e-02,...,1.585979e+06,180428.771377,1.753965e-04,0.014406,84.186317,0.465277,5.615744e-06,1.817294e-04,-1.086543e-11,-5.044664e-11
0,0.000000e+00,0.000000e+00,0.0,0.0,9.994211e+06,10451.324378,-1.382307e-06,0.000007,119.018544,6.082225e-03,...,1.577517e+06,176934.340221,2.961627e-04,0.014392,84.181996,0.456952,-1.779371e-06,1.674434e-04,-7.372970e-11,3.228585e-10
0,0.000000e+00,0.000000e+00,0.0,0.0,9.993274e+06,9473.848196,-1.543777e-06,0.000006,119.019854,1.916549e-03,...,1.595649e+06,183986.562193,2.098396e-04,0.014300,84.203006,0.478596,1.343774e-05,1.748941e-04,7.838632e-11,-4.889444e-11
0,0.000000e+00,0.000000e+00,0.0,0.0,9.992425e+06,8488.752857,-1.291574e-06,0.000006,119.020200,1.421678e-14,...,1.577387e+06,191753.560209,2.359720e-04,0.014062,84.194683,0.508876,-1.607210e-05,2.144586e-04,-3.725290e-11,1.984881e-10
0,0.000000e+00,0.000000e+00,0.0,0.0,9.991727e+06,7652.170966,-1.038462e-06,0.000006,119.020200,1.421678e-14,...,1.583204e+06,188257.219081,1.326336e-04,0.013707,84.144764,0.538046,2.366519e-06,2.196625e-04,-3.880511e-11,-3.213063e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.114871e+07,4909.096785,5.161176e-11,0.000010,117.638607,1.284836e-02,...,1.209924e+07,21268.816202,1.395876e-06,0.000019,66.209437,0.018270,-7.963360e-07,7.946199e-08,3.094850e+26,0.000000e+00
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.114871e+07,4909.096785,5.161176e-11,0.000010,117.637998,1.244450e-02,...,1.209994e+07,21254.775111,-1.978338e-08,0.000020,66.206274,0.018270,-7.962482e-07,7.917201e-08,3.094850e+26,-1.552204e-12
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.114871e+07,4909.096785,5.161176e-11,0.000010,117.636902,1.204096e-02,...,1.209954e+07,21430.806175,-5.367110e-07,0.000021,66.203111,0.018271,-7.964122e-07,7.923202e-08,3.094850e+26,-6.208817e-12
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.114871e+07,4909.096785,5.161176e-11,0.000010,117.635316,1.190659e-02,...,1.209930e+07,21529.622561,-1.762539e-07,0.000021,66.199947,0.018270,-7.964502e-07,7.947666e-08,3.094850e+26,3.104409e-12


In [31]:
# normalize
# Pandas automatically applies colomn-wise function in the code above.
# merged_new_df = (merged_df - merged_df.mean())/merged_df.std()
features = ['P-PDG', 'P-PDG_std','P-PDG_pctchg_mean', 'P-PDG_pctchg_std',
            'P-TPT', 'P-TPT_std', 'P-TPT_pctchg_mean', 'P-TPT_pctchg_std',
            'T-TPT', 'T-TPT_std', 'T-TPT_pctchg_mean', 'T-TPT_pctchg_std',
            'P-MON-CKP', 'P-MON-CKP_std', 'P-MON-CKP_pctchg_mean', 'P-MON-CKP_pctchg_std',
            'T-JUS-CKP', 'T-JUS-CKP_std', 'T-JUS-CKP_pctchg_mean', 'T-JUS-CKP_pctchg_std',
            'pca_1', 'pca_2']

X_train_pre, y_train = merged_df_train.loc[:,features], merged_df_train['class']
X_val_pre, y_val = merged_df_val.loc[:,features], merged_df_val['class']
X_test_pre, y_test = merged_df_test.loc[:,features], merged_df_test['class']

# 각 칼럼의 std는 정규화에서 제외
columns = ['P-PDG', 'P-PDG_pctchg_mean','P-TPT','P-TPT_pctchg_mean', 'T-TPT','T-TPT_pctchg_mean', 'P-MON-CKP','P-MON-CKP_pctchg_mean', 'T-JUS-CKP','T-JUS-CKP_pctchg_mean',
           'P-PDG_std', 'P-TPT_std', 'T-TPT_std', 'P-MON-CKP_std', 'T-JUS-CKP_std',
           'pca_1', 'pca_2']

X_train_pre.loc[:,columns] = (X_train_pre.loc[:,columns] - X_train_pre.loc[:,columns].mean())/X_train_pre.loc[:,columns].std()
X_val_pre.loc[:,columns] = (X_val_pre.loc[:,columns] - X_val_pre.loc[:,columns].mean())/X_val_pre.loc[:,columns].std()
X_test_pre.loc[:,columns] = (X_test_pre.loc[:,columns] - X_test_pre.loc[:,columns].mean())/X_test_pre.loc[:,columns].std()

X_train = X_train_pre
X_val = X_val_pre
X_test = X_test_pre

In [32]:
# training
X_train = X_train[features].values
y_train = y_train.values.astype(int)

X_val = X_val[features].values
y_val = y_val.values.astype(int)

X_test = X_test[features].values
y_test = y_test.values.astype(int)

In [33]:
# options
optimizer_fn = torch.optim.AdamW
optimizer_params = dict(lr=0.0001)
batch_size = 64
patience = 10

In [34]:
print(f'{optimizer_fn.__name__}, {train_ratio}, {val_ratio: .2f}, {test_ratio}, long = {window_size_long} short = {window_size_short}, {overlap_ratio}, lr = {optimizer_params}, batch_size = {batch_size}, no_weigth_loss, patience = {patience}, val_acc = ?, test_acc = ?, no Norm, no Simul')

weight_for_class = []
total_obs = len(y_train)
# check
for key, value in sorted(Counter(y_train).items()):
    print(f'{key} is in event, {value}')
    weight_for_class.append(1 - (value/total_obs))

max_epochs = 200
weight_for_class = torch.Tensor(weight_for_class).type(torch.float32)
print(weight_for_class)

clf = TabNetClassifier(
    optimizer_fn=optimizer_fn,
    optimizer_params=optimizer_params,#1e-2
    # cat_emb_dim = [window_size, window_size, window_size, window_size, window_size]
    )  #TabNetRegressor()

result = clf.fit(
    X_train = X_train,
    y_train = y_train,
    eval_set=[(X_val, y_val)],
    eval_metric = ["accuracy", ],
    loss_fn = nn.CrossEntropyLoss(weight_for_class),
    # loss_fn = nn.CrossEntropyLoss(),
    batch_size = batch_size,
    max_epochs = max_epochs,
    drop_last = True,
    patience = patience
)
# AdamW, 0.7,  0.15, 0.15, long = 1200 short = 1200, 0.95, lr = {'lr': 0.001}, batch_size = 64, no_weigth_loss, patience = 10, val_acc = 0.96924, test_acc = 0.954, no Norm, no Simul
# AdamW, 0.7,  0.15, 0.15, long = 1200 short = 1200, 0.95, lr = {'lr': 0.001}, batch_size = 128, no_weigth_loss, patience = 15, val_acc = .883, test_acc = .957, no Norm, no Simul

AdamW, 0.7,  0.15, 0.15, long = 1200 short = 1200, 0.95, lr = {'lr': 0.0001}, batch_size = 64, no_weigth_loss, patience = 10, val_acc = ?, test_acc = ?, no Norm, no Simul
0 is in event, 6094
4 is in event, 1656
7 is in event, 84
tensor([0.2221, 0.7886, 0.9893])


ValueError: Input contains NaN.

In [37]:
preds = clf.predict(X_test)
print(accuracy_score(preds, y_test))
print(classification_report(y_test, preds))

0.9618039904154113
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     31604
           2       0.00      0.00      0.00        19
           3       0.97      0.96      0.96      1569
           4       0.86      1.00      0.92      8919
           5       0.00      0.00      0.00        30
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00         0

    accuracy                           0.96     42151
   macro avg       0.40      0.42      0.41     42151
weighted avg       0.97      0.96      0.96     42151



In [ ]:
# saving_path_name = './tabnet_test_acc_9904'
# clf.save_model(saving_path_name)